# SACC with clusters
The default SACC scripts in the directory above show how one can make a SACC object for a 3x2 point analysis. The constructor for a SACC object has additional fields for handling clusters. This notebook details how one can use those fields to create/load/split a SACC that has cluster information.

Note: this notebook is for *stacks* of clusters. Individual cluster measurements are not yet supported by SACC.

In [1]:
import sacc

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Cluster stack details
Stacks of clusters are different from regular tracers, since they are binned not only in redshift but also by richness. In this example, we have 20 cluster stacks: 5 bins in richness and 4 tomographic bins. Since this is a tomographic analysis, each cluster stack can be associated with some number of source bins. This association is handled in later cells.

The following two cells create two sets of tracers:
1. cluster stack tracers, that hold tomographic and mass-proxy (aka richness) bin edges
2. source galaxy tracers, that are associated with individual $\gamma_T$ weak lensing profiles for each stack-souce tracer pair

We could also create a new type of tracer for cluster stacks - that would make more sense in general

In [2]:
s = sacc.Sacc()

richness_bin_edges = [20, 30, 50, 80, 120, 180]
source_zbin_centers = [0.5, 0.7, 0.9, 1.1]
cluster_zbin_centers = [0.3, 0.5, 0.7, 0.9]
nbin_richness = len(richness_bin_edges) - 1
nbin_source = len(source_zbin_centers)
nbin_cluster = len(cluster_zbin_centers)

In [3]:
# First we build the cluster stack tracers.
# Here we will store the mass information in metadata,
# but for more complicated data we should write a new
# subclass of Tracer to store this information.

for i, z_mid in enumerate(cluster_zbin_centers):
    z = np.arange(z_mid-0.1, z_mid+0.1, 0.001)
    Nz = np.exp(-(z-z_mid)**2 / (2*0.03**2))

    for j in range(nbin_richness):
        l_min = richness_bin_edges[j]
        l_max = richness_bin_edges[j+1]
        name = f'clusters_{i}_{j}'
        metadata = {'Mproxy_name': 'richness',
                    'Mproxy_min': l_min, 'Mproxy_max': l_max, 
                    'source_name':'lsst_sources'
        }
        s.add_tracer('NZ', name, z, Nz, metadata=metadata)

In [4]:
# Now we move on to the more standard galaxy tracers -
# tomographic LSST source galaxies with 4 redshift bins
for i,z_mid in enumerate(source_zbin_centers):
    # Basic n(z) information
    z = np.arange(z_mid-0.1, z_mid+0.1, 0.001)
    Nz = np.exp(-(z-z_mid)**2 / (2*0.025**2))

    # Some random shapes of Nz to marginalise over
    # We save these as extra columns
    DNz=np.zeros((len(Nz),2))
    DNz[:,0]=(z-z_mid)**2*0.01
    DNz[:,0]-=DNz[:,0].mean()
    DNz[:,1]=(z-z_mid)**3*0.01
    DNz[:,1]-=DNz[:,1].mean()
    extra_columns = {'DNz_0': DNz[:,0], 'DNz_1': DNz[:,1]}

    s.add_tracer("NZ", f"lsst_sources_{i}", z, Nz, extra_columns=extra_columns)

### Data vectors and binning
The SACC holds data vectors and binning information. In this example, we have binning for cluster number counts as well as binning for cluster-source lensing profiles. Both are created in the following cell, as well as the binning information.

In [5]:
# Here we have 10 radial bins for cluster weak lensing
# Note that the "radial bins" can be actual distances or angles on the sky
radii = np.logspace(np.log10(0.5), np.log10(3), 10)

# One of our identifiers is a standard type name that is predefined
cluster_count = sacc.standard_types.cluster_mass_count_wl

# Our other one is manually defined, because it's one of those measurements
# where people pretend to know exactly how physical scale corresponds to angle.
# So we define our own tag for it.
cluster_lensing = "clusterGalaxy_densityShear_xi_tComoving"
# There is a standard format for these names.  We check that we fit it
# by running the parser on it
type_details = sacc.parse_data_type_name(cluster_lensing)
print(type_details.sources, type_details.properties)

for i in range(nbin_cluster):
    for j in range(nbin_richness):
        # Cluster number counts data
        tracer1 = f'clusters_{i}_{j}'

        # random data values.  For now!
        mass= i*1e14
        richness = 5*j
        value = int(np.random.normal((i+10)*100, 100))
        s.add_data_point(cluster_count, (tracer1,), value, err=100.)
        
        # And now the cluster lensing data points
        for k in range(nbin_source):
            tracer2 = f"lsst_sources_{k}"
            # Separate random data values for each point
            for r in radii:
                value = np.random.uniform(0., 10.)
                s.add_data_point(cluster_lensing, (tracer1, tracer2), value, radius=r, err=2.0)

['cluster', 'galaxy'] ['density', 'shear']


### Covariance matrices
Finally, the SACC object holds a covariance matrix between all of the data we use.

In [6]:
n = len(s)
C = np.zeros((n,n))

for i in range(n):
    di = s.data[i]
    for j in range(n):
        dj = s.data[j]
        if i==j and di.data_type == cluster_count:
            C[i,i] = di['err']**2
        elif di.data_type == cluster_lensing:
            C[i,j] = 0.1 * di['err'] * dj['err']
            if i==j:
                C[i,j] *= 10.
        C[j,i] = C[i,j]

s.add_covariance(C)

In [7]:
# This shuffles the data and covariance order so that it is
# organized with all the data points of the same type collected
# together
s.to_canonical_order()

In [8]:
# Add some meta data
s.metadata['Creator'] = 'McGyver'
s.metadata['Project'] = 'Victory'

In [9]:
s.save_fits("clusters.sacc", overwrite=True)

## Loading and splitting
A SACC object with cluster information can be loaded and split, just like the example SACC in the 3x2pt analysis.

In [10]:
s2 = sacc.Sacc.load_fits("./clusters.sacc")
for d in s2.data[:10]:
    print(d)

DataPoint(data_type='cluster_mass_count_wl', tracers=('clusters_0_0',), value=1112, err=100.0)
DataPoint(data_type='cluster_mass_count_wl', tracers=('clusters_0_1',), value=1083, err=100.0)
DataPoint(data_type='cluster_mass_count_wl', tracers=('clusters_0_2',), value=1033, err=100.0)
DataPoint(data_type='cluster_mass_count_wl', tracers=('clusters_0_3',), value=949, err=100.0)
DataPoint(data_type='cluster_mass_count_wl', tracers=('clusters_0_4',), value=1028, err=100.0)
DataPoint(data_type='cluster_mass_count_wl', tracers=('clusters_1_0',), value=1185, err=100.0)
DataPoint(data_type='cluster_mass_count_wl', tracers=('clusters_1_1',), value=1018, err=100.0)
DataPoint(data_type='cluster_mass_count_wl', tracers=('clusters_1_2',), value=1038, err=100.0)
DataPoint(data_type='cluster_mass_count_wl', tracers=('clusters_1_3',), value=1172, err=100.0)
DataPoint(data_type='cluster_mass_count_wl', tracers=('clusters_1_4',), value=985, err=100.0)


In [11]:
# Printing a summary
for dt in s2.get_data_types():
    ind = s2.indices(dt)
    n = len(ind)
    print(f"{dt}: {n} data points")
    for tracers in s2.get_tracer_combinations(dt):
        ind = s2.indices(dt, tracers)
        n = len(ind)
        tracers = '-'.join(tracers)
        print(f"    {tracers}: {n} data points")

cluster_mass_count_wl: 20 data points
    clusters_0_0: 1 data points
    clusters_0_1: 1 data points
    clusters_0_2: 1 data points
    clusters_0_3: 1 data points
    clusters_0_4: 1 data points
    clusters_1_0: 1 data points
    clusters_1_1: 1 data points
    clusters_1_2: 1 data points
    clusters_1_3: 1 data points
    clusters_1_4: 1 data points
    clusters_2_0: 1 data points
    clusters_2_1: 1 data points
    clusters_2_2: 1 data points
    clusters_2_3: 1 data points
    clusters_2_4: 1 data points
    clusters_3_0: 1 data points
    clusters_3_1: 1 data points
    clusters_3_2: 1 data points
    clusters_3_3: 1 data points
    clusters_3_4: 1 data points
clusterGalaxy_densityShear_xi_tComoving: 800 data points
    clusters_0_0-lsst_sources_0: 10 data points
    clusters_0_0-lsst_sources_1: 10 data points
    clusters_0_0-lsst_sources_2: 10 data points
    clusters_0_0-lsst_sources_3: 10 data points
    clusters_0_1-lsst_sources_0: 10 data points
    clusters_0_1-lsst_sou

In [12]:
# Splitting the data into two parts
s3 = s2.copy()
s4 = s2.copy()
s3.keep_selection(cluster_count)
s4.keep_selection(cluster_lensing)
print(len(s3), len(s4))

20 800
